## Statistical modeling

For this example, we'll use the Wisconsin Diagonistic Breast Cancer (WDBC) dataset,
which can be retrieved from the University of California at Irvine Machine Learning
Repository: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/.
Download the file called `wdbc.data` and save it somewhere convenient.

The WDBC dataset consists of 569 observations of 32 variables.
The first column is the patient ID, second is the diagnosis ("M" for malignant,
"B" for benign).
The remaining columns are various measurements of the patients' tumors as determined
by imaging.
Today we'll only consider the first 12 variables:

* Patient ID
* Diagnosis
* Tumor radius
* Texture (standard deviation of the grayscale values in the image)
* Perimeter
* Area
* Smoothness (local variation in radii)
* Compactness (perimeter<sup>2</sup> / area - 1)
* Concavity (severity of concave portions of the contour)
* Number of concave points
* Symmetry
* Fractal dimension (coastline approximation - 1)

### Getting started

First we have to load the DataFrames, GLM, and Gadfly packages.
If you don't already have these installed, you can install them with `Pkg.add`,
e.g. `Pkg.add("DataFrames")`.
If you're using JuliaPro, these are already installed.
Otherwise, Gadfly might take a while to install and compile, so be prepared.

In [ ]:
using DataFrames, GLM, Gadfly

### Reading and manipulating data

Now read the WDBC dataset into a `DataFrame`.
Make sure to modify the path to reflect where you've saved the file.

In [ ]:
wdbc = readtable("/Users/alex/Downloads/wdbc.data", header=false)

Now we'll select the columns of interest out of the 32 and give the columns meaningful
names.
Notice that when we read in the data, the file doesn't have a header to describe the
columns, so Julia automatically generates unique column names.

In [ ]:
# Select all rows (:) and columns 1 to 12 (1:12)
wdbc = wdbc[:, 1:12]

# Modify the dataset in place to apply the names
names!(wdbc, [:id, :diagnosis, :radius, :texture, :perimeter, :area, :smooth,
              :compact, :concavity, :nconcave, :symmetry, :fracdim])

### Fitting a linear regression model

Let's look at how to fit an OLS linear regression model.
We start by plotting the data using a scatter plot, with tumor area on the
x-axis and texture on the y-axis.
For this we'll use the Gadfly package, which provides a Grammar of Graphics
style of specifying plots.

In [ ]:
plot(wdbc, x=:area, y=:texture, Geom.point)

Let's take another look, this time with both axes on the log scale.

In [ ]:
plot(wdbc, x=:area, y=:texture, Geom.point, Scale.x_log, Scale.y_log)

Now we'll try fitting a regression model, predicting the logarithm of the
tumor texture from the logarithm of the area.
First we need to add columns to the dataset corresponding to the transformed
variables, then we can apply the `lm` function in a fashion similar to R.

In [ ]:
# Broadcast log over each element in the given column
wdbc[:log_texture] = log.(wdbc[:texture])
wdbc[:log_area] = log.(wdbc[:area])

# Fit an OLS regression model
# lm(...) is a synonym for fit(LinearModel, ...)
ols = lm(log_texture ~ log_area, wdbc)

We can ask for various properties and diagnostics of the model, like *R*<sup>2</sup>, AIC, etc:

In [ ]:
println("R² = ", r²(ols))
println("AIC = ", aic(ols))

To get the superscript 2 at the Julia REPL, in the Juno IDE, or in a Jupyter Notebook,
type `\^2` and hit <kbd>tab</kbd>.
The Julia plugin for various editors, e.g. Vim and Emacs, also supports this.

We can also look at a plot of the residuals versus the values predicted by the model:

In [ ]:
plot(x=residuals(ols), y=predict(ols), Geom.point,
     Guide.xlabel("Residuals"), Guide.ylabel("Predicted Values"))

### Fitting a logistic regression model

Now let's take a look at modeling the diagnosis, malignant or benign, based on
some of the measurements taken from the imaging.
We'll begin by making a simple box plot of the tumor concavity by diagnosis.

In [ ]:
plot(wdbc, x=:diagnosis, y=:concavity, Geom.boxplot)

In order to perform logistic regression, we need to properly code our dichotomous
response.
If we leave the `diagnosis` column as-is, the GLM package won't know which level
to use as the reference.
To do this, we'll define a new column called `malignant` that contains `Bool` values
denoting malignant diagnoses.
GLM will use `false` as the reference value and model the probability of malignancy.

In [ ]:
wdbc[:malignant] = wdbc[:diagnosis] .== "M"

Fitting the model is similar to the simple linear regression model; this time
we use the `glm` function.
To perform logistic regression, we want to tell GLM that the response is binomial
and that we want logit as the link function, and we can accomplish that simply
by passing in a `Distribution` object from the Distributions package (automatically
installed when installing GLM) and a `LogitLink` object.

In this example, we'll use concavity as the sole predictor.

In [ ]:
glm(malignant ~ concavity, wdbc, Binomial(), LogitLink())

Let's try this again with a more complex model.
This time we'll use concavity, log area, log texture, and the interaction of log area
and log texture.
As in R, model terms are separated by `+`.
`*` is available for specifying main effects and interaction in one fell swoop.
That is, `y ~ x1 * x2` expands to `y ~ x1 + x2 + x1 & x2`, where `&` denotes interaction.
This will be apparent in the formula section of the output Julia provides for models.

In [ ]:
lr = glm(malignant ~ concavity + log_area * log_texture, wdbc, Binomial(), LogitLink())

### Working with probability distributions

Julia has a very natural, generic interface for working with probability distributions
via the Distributions package.
Each distribution has its own type, which permits having a core set of basic APIs that
can be overloaded for any given distribution.
We've already seen an example of this above, where we passed `Binomial()` to `glm`;
`Binomial()` creates an object with type `Binomial` using the default parameters.

Probability distributions can easily be plotted using Gadfly.
As an example, let's plot the PDFs of a bunch of normal distributions, varying only
the variance.

In [ ]:
using Distributions

In [ ]:
# Create an array of anonymous functions, where each simply calls the
# Normal PDF with a different variance, and plot the lot of them over
# the range -10 to 10
plot([x -> pdf(Normal(0, v), x) for v in 1:10], -10.0, 10.0)

The CDF can also be accessed similarly to the PDF using the aptly named `cdf` function.
This makes computing p-values from any distribution quite simple.
For example, say we're running an analysis and we get a $\chi^2$ statistic of 11.5 on
5 degrees of freedom.
We compute the p-value as

In [ ]:
1 - cdf(Chisq(5), 11.5)

The p-value is significant at the 0.05 level and you know what that means: time to publish!

### Filling in the gaps

Sometimes nobody has written the code for a particular hypothesis test or other statistical
task yet, and you'll have to write your own.
This is typically an enormous pain to do in SAS, and navigating the myriad inconsistencies
of the R API can take as much time as actually writing the code.
This kind of thing actually plays well to Julia's strengths.

To illustrate this, let's implement the Mantel-Haenszel test for linear association in
contingency tables with a custom score function.
We'll start with a function that computes midrank scores.
For each marginal total, we add up the previous ones, then add the current + 1 over 2.

In [ ]:
function midrank(table::Matrix, dim::Int)
    # Compute the marginal totals over dimension dim
    totals = sum(table, dim)

    # Compute the midrank scores for the margin
    s = zeros(length(totals))
    @inbounds for i in eachindex(totals)
        s[i] = sum(totals[1:i-1]) + (totals[i] + 1) / 2
    end

    return s
end

Now we'll define a function to compute the correlation based on a given
score function.
The correlation is computed as

$$
r = \frac{\sum_i \sum_j (u_i - \bar{u}) \, (v_j - \bar{v}) \, n_{ij}}{\sqrt{\left[ \sum_i \sum_j (u_i - \bar{u})^2 \, n_{ij} \right] \left[ \sum_i \sum_j (v_j - \bar{v})^2 \, n_{ij} \right]}}
$$

where the *u*<sub>*i*</sub> are the row scores, *v*<sub>*j*</sub> are the column scores,
and *n*<sub>*ij*</sub> are the entries in the contingency table.

In [ ]:
# A custom correlation function that supports multiple kinds of scores
# Sums are written as matrix multiplications where applicable

function correlation(X::Matrix, score::Symbol=:integer)
    if score === :integer
        u = collect(1:size(table, 1))
        v = collect(1:size(table, 2))
    elseif score === :midrank
        u = midrank(table, 2)
        v = midrank(table, 1)
    else
        throw(ArgumentError("unrecognized scoring method `$score`"))
    end

    n = sum(X)
    ubar = sum(u'X) / n
    vbar = sum(X*v) / n
    numer = (u .- ubar)' * X * (v .- vbar)
    denom1 = sum(((u .- ubar).^2)'X)
    denom2 = sum(X * (v .- vbar).^2)
    
    # numer is a one-element matrix, so we can unwrap it to get the
    # value using first
    return first(numer) / sqrt(denom1 * denom2)
end

The test statistic is computed as

$$
M^2 = (n - 1) \, r^2
$$

where *n* is the total sample size and *r* is the correlation.
Under the null, this has an approximate $\chi^2$ distribution with 1 degree of freedom.

In [ ]:
# Compute the M² statistic and the p-value

function mantelhaenszel(table::Matrix, score::Symbol=:integer)
    r = correlation(table, score)
    h = (sum(table) - 1) * r^2
    return (h, 1 - cdf(Chisq(1), h))
end

To test our functions, say we have the following contingency table of tonsil
enlargement by streptococcus pyogenes carrier status.

| Status      | Normal | Slight | Severe |
| ----------- |-------:|-------:|-------:|
| Carrier     | 19     | 29     | 24     |
| Non-carrier | 497    | 560    | 269    |

Then we can carry out our tests like so:

In [ ]:
table = [ 19  29  24;
         497 560 269]

for score in (:integer, :midrank)
    println("Scoring method: $score")

    println("Correlation: ", correlation(table, score))

    statistic, pvalue = mantelhaenszel(table, score)
    println("Mantel-Haenszel: ", statistic, " with p=", pvalue)

    println()
end

Tasks like these can be surprisingly easy in Julia!

### Taking it further

For more information on the capabilities of the packages mentioned here,
take a look at the documentation:

* DataFrames: https://juliastats.github.io/DataFrames.jl/stable/
* Gadfly: http://gadflyjl.org/stable/
* GLM: https://github.com/JuliaStats/GLM.jl/blob/master/README.md

There's also the Discourse forum for asking questions, which includes a
tags for statistics, bio, data, visualization, and many more:
https://discourse.julialang.org/.